In [ ]:
import yfinance as yf
import numpy as np

df = yf.download('^GSPC', start='1980-01-01')

for period in [20, 50, 100, 200]:
    df[f'SMA_{period}'] = df['Close'].rolling(period).mean()
    df[f'EMA_{period}'] = df['Close'].ewm(span=period, adjust=False).mean()

ema12 = df['Close'].ewm(span=12, adjust=False).mean()
ema26 = df['Close'].ewm(span=26, adjust=False).mean()
df['MACD'] = ema12 - ema26
df['MACD_signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
df['MACD_hist'] = df['MACD'] - df['MACD_signal']

delta = df['Close'].diff()
gain = delta.clip(lower=0)
loss = -delta.clip(upper=0)
avg_gain = gain.rolling(14).mean()
avg_loss = loss.rolling(14).mean()
rs = avg_gain / avg_loss
df['RSI_14'] = 100 - (100 / (1 + rs))

df['ROC_14'] = ((df['Close'] / df['Close'].shift(14)) - 1) * 100

high_low = df['High'] - df['Low']
high_pc = abs(df['High'] - df['Close'].shift(1))
low_pc = abs(df['Low'] - df['Close'].shift(1))
df['TR'] = np.maximum(high_low, np.maximum(high_pc, low_pc))

df['+DM'] = np.where(
    (df['High'].diff() > -df['Low'].diff(-1)) & 
    (df['High'].diff() > 0),
    df['High'].diff(), 0
)
df['-DM'] = np.where(
    (-df['Low'].diff(-1) > df['High'].diff()) & 
    (df['Low'].diff() < 0),
    -df['Low'].diff(), 0
)

def wilder_smooth(series, window):
    return series.ewm(alpha=1/window, adjust=False).mean()

df['TR_14'] = wilder_smooth(df['TR'], 14)
df['+DI_14'] = (wilder_smooth(df['+DM'], 14) / df['TR_14']) * 100
df['-DI_14'] = (wilder_smooth(df['-DM'], 14) / df['TR_14']) * 100
df['DX'] = (abs(df['+DI_14'] - df['-DI_14']) / (df['+DI_14'] + df['-DI_14'])) * 100
df['ADX'] = wilder_smooth(df['DX'], 14)

df['lowest_low_14'] = df['Low'].rolling(14).min()
df['highest_high_14'] = df['High'].rolling(14).max()

df['temp_denom'] = df['highest_high_14'] - df['lowest_low_14']
df['%K'] = 0.0
mask = df['temp_denom'] != 0  
df.loc[mask, '%K'] = (df.loc[mask, 'Close'] - df.loc[mask, 'lowest_low_14']) / df.loc[mask, 'temp_denom'] * 100
df['%D'] = df['%K'].rolling(3).mean()

df['%R'] = 0.0
df.loc[mask, '%R'] = (df.loc[mask, 'highest_high_14'] - df.loc[mask, 'Close']) / df.loc[mask, 'temp_denom'] * -100

df.drop(['TR', '+DM', '-DM', 'TR_14', 'DX', 
         'lowest_low_14', 'highest_high_14', 'temp_denom'], axis=1, inplace=True)

df